<a href="https://colab.research.google.com/github/spetznick/applied-gpu-programming/blob/colab-s/assignment-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Assignment 3 Exercise 2



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive/DD2360/DD2360-PIC")

In [3]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Fri Dec 13 14:14:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  7

## We compile the saved cuda code using nvcc compiler

In [8]:
!pwd
!make clean
!make

/content/drive/MyDrive/DD2360/DD2360-PIC
rm -rf  src/ConfigFile.o src/EMfield.o src/EMfield_aux.o src/Particles_aux.o src/Timing.o src/RW_IO.o src/TransArraySpace3D.o src/InterpDensNet.o src/Grid.o src/InterpDensSpecies.o src/BC.o src/Basic.o src/sputniPIC.o src/Particles.o
rm -rf sputniPIC.out
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/ConfigFile.cpp -c -o src/ConfigFile.o
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/EMfield.cpp -c -o src/EMfield.o
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/EMfield_aux.cpp -c -o src/EMfield_aux.o
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O3 -g -Xcompiler -Wall src/Particles_aux.cpp -c -o src/Particles_aux.o
src/Particles_aux.cpp: In function ‘void particle_aux_allocate(particles*, particles_aux*, int)’:
src/Particles_aux.cpp:14:10: warning: unused variable ‘npmax’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wu

In [ ]:
!./bin/sputnicPIC.out ./inputfiles/GEM_2D.inp